# Build RAG pipeline using Open Source Large Languages

In the notebook we will build a Chat with Website use cases using Zephyr 7B model

## Installation

In [1]:
!pip install langchain faiss-cpu sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## Import RAG components required to build pipeline

In [2]:
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA, LLMChain

## Setup HuggingFace Access Token

- Log in to [HuggingFace.co](https://huggingface.co/)
- Click on your profile icon at the top-right corner, then choose [“Settings.”](https://huggingface.co/settings/)
- In the left sidebar, navigate to [“Access Token”](https://huggingface.co/settings/tokens)
- Generate a new access token, assigning it the “write” role.


In [4]:
import os
from getpass import getpass

HF_TOKEN = getpass("HF Token:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

HF Token:··········


## External data/document - ETL

In [59]:
import nest_asyncio

nest_asyncio.apply()

In [60]:
WEBSITE_URL = "https://simple.wikipedia.org/wiki/Main_Page"

In [61]:
loader = WebBaseLoader(WEBSITE_URL)
loader.requests_per_second = 1
docs = loader.aload()

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 10.59it/s]


In [62]:
docs

[Document(page_content='\n\n\n\nWikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tGetting around\n\t\n\n\nMain pageSimple startSimple talkNew changesShow any pageHelpContact usGive to WikipediaAbout Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMain Page\n\n\n\n\n\n\n\n\n\n\nMain PageTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nReadView sourceView history\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadView sourceView history\n\n\n\n\n\n\t\tGeneral\n\t\n\n\nWhat links hereRelated chan

## Text Splitting - Chunking

In [63]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

In [64]:
chunks[0]

Document(page_content='Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tGetting around\n\t\n\n\nMain pageSimple startSimple talkNew changesShow any pageHelpContact usGive to WikipediaAbout Wikipedia', metadata={'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia', 'language': 'en'})

In [65]:
chunks[1]

Document(page_content='Search\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMain Page', metadata={'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia', 'language': 'en'})

In [66]:
chunks[2]

Document(page_content='Main Page\n\n\n\n\n\n\n\n\n\n\nMain PageTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nReadView sourceView history\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadView sourceView history\n\n\n\n\n\n\t\tGeneral', metadata={'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia', 'language': 'en'})

In [67]:
chunks[3]

Document(page_content='General\n\t\n\n\nWhat links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageGet shortened URLDownload QR codeWikidata item\n\n\n\n\n\n\t\tPrint/export\n\t\n\n\nMake a bookDownload as PDFPage for printing', metadata={'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia', 'language': 'en'})

In [68]:
chunks[4]

Document(page_content='In other projects\n\t\n\n\nWikimedia CommonsWikimedia Foundation Governance WikiMediaWikiMeta-WikiWikimedia OutreachMultilingual WikisourceWikispeciesWikidataWikifunctionsWikimaniaWiktionary', metadata={'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia', 'language': 'en'})

## Embeddings

In [69]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

## Vector Store - FAISS or ChromaDB

In [70]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [71]:
vectorstore

In [72]:
query = "how mant number of mourning doves?"
search = vectorstore.similarity_search(query)

In [73]:
search[0].page_content

'The mourning dove (Zenaida macroura) is a member of the dove family (Columbidae). It has five subspecies. The number of mourning doves is about 475 million. They live in North America. Mourning doves are light grey and brown, and males and females look'

## Retriever

In [74]:
retriever = vectorstore.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
)

In [75]:
retriever.get_relevant_documents(query)

[Document(page_content='The mourning dove (Zenaida macroura) is a member of the dove family (Columbidae). It has five subspecies. The number of mourning doves is about 475 million. They live in North America. Mourning doves are light grey and brown, and males and females look', metadata={'language': 'en', 'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia'}),
 Document(page_content='People hunt mourning doves for sport and for meat. Up to 70 million birds are shot in the United States every year. Its name, "mourning," comes from its sad-sounding call. The bird is a strong flier, and can fly at up to 88\xa0km/h (55\xa0mph).', metadata={'language': 'en', 'source': 'https://simple.wikipedia.org/wiki/Main_Page', 'title': 'Wikipedia'}),
 Document(page_content='These birds usually have one partner at a time. Both parents sit on the eggs and care for their chicks. Adult mourning doves usually eat only seeds. The parents feed crop milk to the young.', metadata={'langu

## Large Language Model - Open Source

In [76]:
llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

## Prompt Template and User Input (Augment - Step 2)

In [87]:
query = "how many number of mourning doves?"

prompt = f"""
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

## RAG RetrievalQA chain

In [78]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)

In [79]:
response = qa.run(prompt)

In [80]:
response

'The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n how mant number of mourning doves?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n how mant number of mourning doves?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n how mant number of mourning doves?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: Context information is below. \n------------\nThe mourning dove (Zenaida macroura) is a member of the dove family (Columbidae). It has five subspecies. The number of

## Chain

In [81]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [82]:
template = """
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

In [83]:
prompt = ChatPromptTemplate.from_template(template)

In [84]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [88]:
response = rag_chain.invoke("how many number of mourning doves?")

In [89]:
print(response)

Human: 
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 how many number of mourning doves?
 </s>
 <|assistant|>
There are no specific numbers of mourning doves as they are a type of bird and their population varies based on different factors such as habitat, predators, and human activities. The global population of mourning doves is estimated to be around 350 million individuals. However, local populations may vary significantly.
